In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.stats import multivariate_normal

# Mean and covariance of bivariate normal
mu = np.array([0, 0])
cov = np.array([[1.0, 0.8], [0.8, 1.0]])

# Grid for surface
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))
rv = multivariate_normal(mean=mu, cov=cov)
Z = rv.pdf(pos)

# Setup figure and 3D axis
fig = plt.figure(figsize=(10, 7), facecolor='white')
ax = fig.add_subplot(111, projection='3d', facecolor='white')
ax.grid(False)
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')

# Surface plot (joint PDF)
surf = ax.plot_surface(X, Y, Z, cmap='plasma', alpha=0.85, edgecolor='none')

# Initialize slice line
slice_line, = ax.plot([], [], [], color='black', linewidth=3)

# Axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Density')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(0, Z.max())

# Animation frame values
x_vals = np.linspace(-2.5, 2.5, 60)

def update(frame):
    x_fixed = np.round(x_vals[frame], 2)
    
    # Compute conditional parameters
    mu_x, mu_y = mu
    sigma_xx = cov[0, 0]
    sigma_yy = cov[1, 1]
    sigma_xy = cov[0, 1]

    cond_mu_y = mu_y + sigma_xy / sigma_xx * (x_fixed - mu_x)
    cond_var_y = sigma_yy - sigma_xy**2 / sigma_xx

    # Slice values
    y_vals = np.linspace(-3, 3, 100)
    z_vals = multivariate_normal.pdf(
        np.column_stack((np.full_like(y_vals, x_fixed), y_vals)),
        mean=mu, cov=cov
    )

    # Update slice line
    slice_line.set_data(np.full_like(y_vals, x_fixed), y_vals)
    slice_line.set_3d_properties(z_vals)

    # Update camera view
    ax.view_init(elev=30 + 5 * np.sin(frame / 10), azim=45 + frame * 2)
    ax.set_title(f'Conditional Slice at X = {x_fixed}', fontsize=14)
    return slice_line,

# Create and save animation
ani = FuncAnimation(fig, update, frames=len(x_vals), interval=100)
ani.save("bivariate_conditional_slice_contrast.gif", writer=PillowWriter(fps=10))
print("✅ GIF saved as 'bivariate_conditional_slice_contrast.gif'")
